import numpy as np
from matplotlib import colors


import scipy.interpolate as intp
import scipy.integrate as intg
from pyrite import _utils as ut
from pyrite.geoid import GeoidHeight as gh
import scipy.io
from mpl_toolkits.basemap import Basemap
import h5py
import time
import sys
import matplotlib.pyplot as plt

from scipy.interpolate import griddata
import scipy.interpolate as intp
from scipy.optimize import leastsq
from scipy.stats.stats import pearsonr

from pyrite import elevation_models
from pyrite import _utils as ut
from pykrige import OrdinaryKriging
from pykrige import variogram_models

from scipy.interpolate import RegularGridInterpolator as RGI
from scipy import interpolate

from mpl_toolkits.mplot3d import Axes3D

import pyproj

model_dict = {'linear': elevation_models.linear_elevation_model,
                      'onn': elevation_models.onn_elevation_model,
                      'onn_linear': elevation_models.onn_linear_elevation_model,
                      'exp': elevation_models.exp_elevation_model,
                      'exp_linear': elevation_models.exp_linear_elevation_model}

residual_dict = {'linear': elevation_models.residuals_linear,
                      'onn': elevation_models.residuals_onn,
                      'onn_linear': elevation_models.residuals_onn_linear,
                      'exp': elevation_models.residuals_exp,
                      'exp_linear': elevation_models.residuals_exp_linear}

initial_dict = {'linear': elevation_models.initial_linear,
                      'onn': elevation_models.initial_onn,
                      'onn_linear': elevation_models.initial_onn_linear,
                      'exp': elevation_models.initial_exp,
                      'exp_linear': elevation_models.initial_exp_linear}

para_numb_dict = {'linear': 2,
                  'onn' : 3,
                  'onn_linear':4,
                  'exp':2,
                  'exp_linear':3}


variogram_dict = {'linear': variogram_models.linear_variogram_model,
                      'power': variogram_models.power_variogram_model,
                      'gaussian': variogram_models.gaussian_variogram_model,
                      'spherical': variogram_models.spherical_variogram_model,
                      'exponential': variogram_models.exponential_variogram_model,
                      'hole-effect': variogram_models.hole_effect_variogram_model}
def ocean_mask(latlist,lonlist):
    row,col = latlist.shape
    mask = np.zeros((row,col))
    for i in range(row):
        for j in range(col):
            lat0 = latlist[i,j]
            lon0 = lonlist[i,j]
            if bm.is_land(lon0, lat0):
                mask[i,j] = 1
            
    return mask

def get_geoid(lat,lon):
    
    row,col = lat.shape
    lat = np.asarray(lat)
    lon = np.asarray(lon)
    lat0 = lat.flatten()
    lon0 = lon.flatten()  
    obj = gh()
    geoidH = []
    for i in range(len(lat0)):
        N0 = gh.get(obj, lat0[i], lon0[i], cubic=True)
        geoidH.append(N0)    
    geoidH = np.asarray(geoidH)
    geoidH = geoidH.reshape(row,col)
    return geoidH


def get_uLOS_llh(lat0,lon0,orb_state_file,width,length,ref_lat1,ref_lon1,ref_lat2,ref_lon2,ref_lat3,ref_lon3):
    
    Orb = read_txt2array(orb_state_file)
    t_orb = Orb[:,1]
    t_tot = Orb[0,2]
    R_ear = Orb[0,3]
    
    X_Orb = Orb[:,4]
    Y_Orb = Orb[:,5]
    Z_Orb = Orb[:,6]
    
    t0 = az0/length*t_tot
    
    tck = intp.interp1d(t_orb,X_Orb,kind='cubic')
    X0 = tck(t0)      #Again negative for consistency with hx
    
    tck = intp.interp1d(t_orb,Y_Orb,kind='cubic')
    Y0 = tck(t0)      #Again negative for consistency with hx
    
    tck = intp.interp1d(t_orb,Z_Orb,kind='cubic')
    Z0 = tck(t0)      #Again negative for consistency with hx
    
    
    SatCoord = (X0,Y0,Z0)
    GroundLLH = (lat0,lon0,0)
    uLOS,uLOS0 = get_uLOS(SatCoord,GroundLLH)
    
    xg,yg,zg = uLOS0
    
    sh = np.sqrt(X0**2 + Y0**2 + Z0**2) - R_ear
    sg = np.sqrt(xg**2 + yg**2 + zg**2)
    
    inc = np.arccos(sh/sg)

    
    
    
    return inc

def lla_to_ecef(lat, lon, alt):
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    x, y, z = pyproj.transform(lla, ecef, lon, lat, alt, radians=False)
    return x, y, z

def ecef_to_lla(X, Y, Z):
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    lon, lat, alt = pyproj.transform(ecef, lla, X, Y, Z, radians=False)
    return lon, lat, alt


def get_uLOS(SatCoord,GroundLLH):
    # GroundLLH = (lat, lon, alt)
    lat, lon, alt = GroundLLH
    
    sX,sY,sZ = SatCoord
    gX,gY,gZ = lla_to_ecef(lat, lon, alt)
    
    uLOS0 = ((sX - gX),(sY - gY),(sZ - gZ))
    dLOS = np.sqrt((sX - gX)**2 + (sY - gY)**2 + (sZ - gZ)**2)
    uLOS = uLOS0/dLOS
    
    return uLOS,uLOS0

def func_trend(lat,lon,p):
    a0,b0,c0,d0 = p
    
    return a0 + b0*lat + c0*lon +d0*lat*lon

def residual_trend(p,lat,lon,y0):
    a0,b0,c0,d0 = p 
    return y0 - func_trend(lat,lon,p)

def func_trend_model(lat,lon,p):
    lat = lat/180*np.pi
    lon = lon/180*np.pi  
    lon = lon*np.cos(lat) # to get isometrics coordinates
    a0,b0,c0,d0 = p
    
    return a0 + b0*lat + c0*lon +d0*lat*lon

def read_txt2array(txt):
    A = np.loadtxt(txt,dtype=np.float32)
    if np.array(A).size ==1:
        A = [A]
    if isinstance(A[0],bytes):
        A = A.astype(str)
    #A = list(A)    
    return A

def latlon2rgaz_coarse(lat0,lon0,attr):
    
    width = int(attr['WIDTH'])
    length = int(attr['LENGTH'])
    ref_lat1 = float(attr['LAT_REF1'])
    ref_lat2 = float(attr['LAT_REF2'])
    ref_lat3 = float(attr['LAT_REF3'])
    ref_lon1 = float(attr['LON_REF1'])
    ref_lon2 = float(attr['LON_REF2'])
    ref_lon3 = float(attr['LON_REF3'])
    dl_Lat =(ref_lat3 - ref_lat1)/length 
    dw_Lat = (ref_lat2 - ref_lat1)/width 
    
    dl_Lon = (ref_lon3 - ref_lon1)/length 
    dw_Lon = (ref_lon2 - ref_lon1)/width 
    
    A = np.zeros((2,2))
    A[0,:] = [dl_Lat,dw_Lat]
    A[1,:] = [dl_Lon,dw_Lon]
    
    
    L = [(lat0 - ref_lat1), (lon0 - ref_lon1)]
    L =np.asarray(L)
    L = L.reshape((2,1))
    
    kk = np.dot(np.linalg.inv(A),L)
    #print(kk)
    azimuth0 = round(kk[0,0] + 1) 
    range0 = round(kk[1,0] + 1) 
    
    return range0,azimuth0



def orb_state_lalo_uLOS(lat0,lon0,orb_state_file,attr):
    # orb_state_file from read_orb_state.py    
    width = int(attr['WIDTH'])
    length = int(attr['LENGTH'])
    ref_lat1 = float(attr['LAT_REF1'])
    ref_lat2 = float(attr['LAT_REF2'])
    ref_lat3 = float(attr['LAT_REF3'])
    ref_lon1 = float(attr['LON_REF1'])
    ref_lon2 = float(attr['LON_REF2'])
    ref_lon3 = float(attr['LON_REF3'])
    ra0,az0 = latlon2rgaz_coarse(lat0,lon0,attr)
    Orb = read_txt2array(orb_state_file)
    t_orb = Orb[:,1]
    t_tot = Orb[0,2]
    R_ear = Orb[0,3]
    
    X_Orb = Orb[:,4]
    Y_Orb = Orb[:,5]
    Z_Orb = Orb[:,6]
    
    t0 = az0/length*t_tot
    
    tck = intp.interp1d(t_orb,X_Orb,kind='cubic',bounds_error =False, fill_value='extrapolate')
    X0 = tck(t0)      #Again negative for consistency with hx
    
    tck = intp.interp1d(t_orb,Y_Orb,kind='cubic',bounds_error =False, fill_value='extrapolate')
    Y0 = tck(t0)      #Again negative for consistency with hx
    
    tck = intp.interp1d(t_orb,Z_Orb,kind='cubic',bounds_error =False, fill_value='extrapolate')
    Z0 = tck(t0)      #Again negative for consistency with hx
    
    
    SatCoord = (X0,Y0,Z0)
    hN = get_geoid_point(lat0,lon0)
    GroundLLH = (lat0,lon0, -hN)
    uLOS,uLOS0 = get_uLOS(SatCoord,GroundLLH)
      
    X1,Y1,Z1 = lla_to_ecef(lat0, lon0, -hN)
    
    xg,yg,zg = uLOS0   
    sh0 = np.sqrt(X0**2 + Y0**2 + Z0**2) - R_ear
    sh = np.sqrt(X0**2 + Y0**2 + Z0**2) 
    sg = np.sqrt(xg**2 + yg**2 + zg**2)
    gv = np.sqrt(X1**2 + Y1**2 + Z1**2)
    
    kk0 = (sg**2 + sh**2 - gv**2)/(2*sg*sh)
    
    inc = np.arccos(kk0)
    #inc = np.arccos(sh0/sg)
    inc = inc/np.pi*180
    
    return uLOS, inc

def remove_ramp(lat,lon,data):
    # mod = a*x + b*y + c*x*y
    lat = lat/180*np.pi
    lon = lon/180*np.pi  
    lon = lon*np.cos(lat) # to get isometrics coordinates
    
    p0 = [0.0001,0.0001,0.0001,0.0000001]
    plsq = leastsq(residual_trend,p0,args = (lat,lon,data))
    para = plsq[0]
    data_trend = data - func_trend(lat,lon,para)
    corr, _ = pearsonr(data, func_trend(lat,lon,para))
    return data_trend, para, corr

def get_geoid_point(lat,lon):
    obj = gh()
    geoidH = []
    N0 = gh.get(obj, lat, lon, cubic=True)
    return N0

def read_attr(fname):
    # read hdf5
    with h5py.File(fname, 'r') as f:
        atr = dict(f.attrs)
        
    return atr

def get_LOS3D_coords(latlist,lonlist,hgt_lvs, orb_state_file,attr):
    
    width = int(attr['WIDTH'])
    length = int(attr['LENGTH'])
    ref_lat1 = float(attr['LAT_REF1'])
    ref_lat2 = float(attr['LAT_REF2'])
    ref_lat3 = float(attr['LAT_REF3'])
    ref_lon1 = float(attr['LON_REF1'])
    ref_lon2 = float(attr['LON_REF2'])
    ref_lon3 = float(attr['LON_REF3'])
    
    row,col = latlist.shape
    nh = len(hgt_lvs)
    
    lat_intp = np.zeros((row,col,nh))
    lon_intp = np.zeros((row,col,nh))
    los_intp = np.zeros((row,col,nh))
    
    #latlist = latlist.flatten()
    #lonlist = lonlist.flatten()
    #nn = len(latlist)

    for i in range(row):
        ut.print_progress(i+1, row, prefix='LOS locations calculating:', suffix='complete', decimals=1, barLength=50, elapsed_time=None)
        for j in range(col):   
            lat0 = latlist[i,j]
            lon0 = lonlist[i,j]
            uLOS, inc = orb_state_lalo_uLOS(lat0,lon0,orb_state_file,attr)
            losX,losY,losZ, latH, lonH, losH = get_uLOS_llh(lat0,lon0,hgt_lvs,uLOS)
            lat_intp[i,j,:] = latH
            lon_intp[i,j,:] = lonH
            los_intp[i,j,:] = losH
    
    return lat_intp, lon_intp, los_intp

def get_uLOS_llh(lat0,lon0,hgt_lvs,uLOS):
    
    hgt_lvs = sorted(hgt_lvs)
    #print(hgt_lvs)
    nn = len(hgt_lvs)
    deltaH = np.asarray(hgt_lvs[1:nn]) - np.asarray(hgt_lvs[0:(nn-1)])
 
    minh = min(hgt_lvs)    # hgt_lvs[0]
    #inc = inc/180*np.pi
    
    gX,gY,gZ = lla_to_ecef(lat0, lon0, minh)
    gX1,gY1,gZ1 = lla_to_ecef(lat0, lon0, minh+1000)
    uZTD = ((gX1-gX)/1000,(gY1-gY)/1000,(gZ1-gZ)/1000)
    
    dH0 = np.sqrt((uLOS[0]*uZTD[0])**2 + (uLOS[1]*uZTD[1])**2 + (uLOS[2]*uZTD[2])**2)
    dH = uLOS[0]*uZTD[0] + uLOS[1]*uZTD[1] + uLOS[2]*uZTD[2]   
    #dLOS_lvs0 = deltaH/np.cos(inc)   

    losX = np.zeros((nn,1));losY = np.zeros((nn,1));losZ = np.zeros((nn,1))
    latH = np.zeros((nn,));lonH = np.zeros((nn,));losH = np.zeros((nn,))
    
    losX[0] = gX;   losY[0]=gY;     losZ[0]=gZ
    latH[0] = lat0; lonH[0] = lon0; losH[0]=0
    
    N = len(deltaH)
    dLOS_lvs = np.zeros((N,1))
    for i in range(N):
        dLOS_lvs[i] = deltaH[i]/dH
        losX[i+1] = losX[i] + uLOS[0]*dLOS_lvs[i]
        losY[i+1] = losY[i] + uLOS[1]*dLOS_lvs[i]
        losZ[i+1] = losZ[i] + uLOS[2]*dLOS_lvs[i]
        lo, la, alt = ecef_to_lla(losX[i],losY[i],losZ[i])
        latH[i+1] = la
        lonH[i+1] = lo
        losH[i+1] = losH[i] + dLOS_lvs[i]
    
    return losX,losY,losZ, latH, lonH, losH


def get_LOS_parameters(latlist,lonlist,Presi,Tempi,Vpri,latH, lonH):
    row,col,nh = Presi.shape
    latv = latlist[:,0]
    lonv = lonlist[0,:]
    
    lat00 = latlist.flatten()
    lon00 = lonlist.flatten()
    
    LosP = np.zeros((row,col,nh))
    LosT = np.zeros((row,col,nh))
    LosV = np.zeros((row,col,nh))
    
    
    def resi_func(m,d,y):
        variogram_function =variogram_dict['spherical'] 
        return  y - variogram_function(m,d)
    
    for i in range(nh):
        ut.print_progress(i+1, nh, prefix='LOS parameters calculating:', suffix='complete', decimals=1, barLength=50, elapsed_time=None)
        lat = latH[:,:,i]
        lon = lonH[:,:,i]
        lat = lat.flatten()
        lon = lon.flatten()
        
        Presi0 = Presi[:,:,i]
        Tempi0 = Tempi[:,:,i]
        Vpri0 = Vpri[:,:,i]
        Vpri0 = Vpri0.flatten()
        
        Vpri0_cor, para, corr= remove_ramp(lat00,lon00,Vpri0)
        trend = func_trend_model(lat,lon,para)
        
        uk = OrdinaryKriging(lon, lat, Vpri0_cor, coordinates_type = 'geographic', nlags=50)
        Semivariance_trend = 2*(uk.semivariance)    
        x0 = (uk.lags)/180*np.pi*R
        y0 = Semivariance_trend
        max_length = 300
        range0 = max_length/2
        LL0 = x0[x0< max_length]
        SS0 = y0[x0< max_length]
        sill0 = max(SS0)
        sill0 = sill0.tolist()

        p0 = [sill0, range0, 0.0001]    
        vari_func = variogram_dict['spherical']        
        tt, _ = leastsq(resi_func,p0,args = (LL0,SS0))   
        corr, _ = pearsonr(SS0, vari_func(tt,LL0))
        if tt[2] < 0:
            tt[2] =0
        para = tt
        para[1] = para[1]/R/np.pi*180

        uk.variogram_model_parameters = para
        z0,s0 = uk.execute('points', lon, lat, n_closest_points = 15, backend='loop')
        z0 = z0 + trend
        LosV[:,:,i] = z0.reshape((latlist.shape))
        ######### interp template and pressure
        Presi0 = Presi0.reshape((latlist.shape))
        Tempi0 = Tempi0.reshape((latlist.shape))

        fP = RGI((latv[::-1], lonv), Presi0[::-1,:],method='linear',bounds_error=False,fill_value=None)
        fT = RGI((latv[::-1], lonv), Tempi0[::-1,:],method='linear',bounds_error=False,fill_value=None)

        points = np.zeros((len(lat),2))
        points[:,0] = lat
        points[:,1] = lon
        Pintp = fP(points)
        Tintp = fT(points)
        LosT[:,:,i] = Tintp.reshape((latlist.shape))
        LosP[:,:,i] = Pintp.reshape((latlist.shape))
    
    return LosP,LosT,LosV


def losPTV2del(Presi,Tempi,Vpri,losHgt,cdict,verbose=False):
    '''Computes the delay function given Pressure, Temperature and Vapor pressure.

    Args:
        * Presi (np.array) : Pressure at height levels.
        * Tempi (np.array) : Temperature at height levels.
        * Vpri  (np.array) : Vapor pressure at height levels.
        * hgt   (np.array) : Height levels.
        * cdict (np.array) : Dictionary of constants.

    Returns:
        * DDry2 (np.array) : Dry component of atmospheric delay.
        * DWet2 (np.array) : Wet component of atmospheric delay.

    .. note::
        Computes refractive index at each altitude and integrates the delay using cumtrapz.'''

    if verbose:
        print('PROGRESS: COMPUTING DELAY FUNCTIONS')
    nhgt = Presi.shape[2]           #Number of height points
    nlat = Presi.shape[0]        #Number of stations
    nlon = Presi.shape[1]
    DryT = Presi/Tempi
    WonT = Vpri/Tempi
    WonT2 = WonT/Tempi

    k1 = cdict['k1']
    Rd = cdict['Rd']
    Rv = cdict['Rv']
    k2 = cdict['k2']
    k3 = cdict['k3']
    g = cdict['g']

    #Dry delay
    DDry2 = np.zeros((nlat,nlon,nhgt))
    #DDry2[:,:,:] = k1*Rd*(Presi[:,:,:] - Presi[:,:,-1][:,:,np.newaxis])*1.0e-6/g
    for i  in range(nlat):
        for j in range(nlon):
            DDry2[i,j,:] = intg.cumtrapz(DryT[i,j,:],x=losHgt[i,j,:],axis=-1)
    
    #val = 2*DDry2[:,:,-1]-DDry2[:,:,-2]
    #val = val[:,:,None]
    #DDry2 = np.concatenate((DDry2,val),axis=-1)
    
    DDry2 = -1.0e-6*k1*DDry2
    for i in range(nlat):
        for j in range(nlon):
            DDry2[i,j,:]  = DDry2[i,j,:] - DDry2[i,j,-1]

    #Wet delay
    S1 = np.zeros((nlat,nlon,nhgt-1))
    for i  in range(nlat):
        for j in range(nlon):
            S1[i,j,:] = intg.cumtrapz(WonT[i,j,:],x=losHgt[i,j,:],axis=-1)
            
    #S1 = intg.cumtrapz(WonT,x=hgt,axis=-1)
    val = 2*S1[:,:,-1]-S1[:,:,-2]
    val = val[:,:,None]
    S1 = np.concatenate((S1,val),axis=-1)
    del WonT
    
    S2 = np.zeros((nlat,nlon,nhgt-1))
    for i  in range(nlat):
        for j in range(nlon):
            S2[i,j,:] = intg.cumtrapz(WonT2[i,j,:],x=losHgt[i,j,:],axis=-1)
    #S2 = intg.cumtrapz(WonT2,x=hgt,axis=-1)
    val = 2*S2[:,:,-1]-S2[:,:,-2]
    val = val[:,:,None]
    S2 = np.concatenate((S2,val),axis=-1)
    DWet2 = -1.0e-6*((k2-k1*Rd/Rv)*S1+k3*S2)
    
    for i in range(nlat):
        for j in range(nlon):
            DWet2[i,j,:]  = DWet2[i,j,:] - DWet2[i,j,-1]

    return DDry2,DWet2

def get_sar_area(attr):
    ref_lat1 = float(attr['LAT_REF1'])
    ref_lat2 = float(attr['LAT_REF2'])
    ref_lat3 = float(attr['LAT_REF3'])
    ref_lon1 = float(attr['LON_REF1'])
    ref_lon2 = float(attr['LON_REF2'])
    ref_lon3 = float(attr['LON_REF3'])
    
    maxlon = max((ref_lon1,ref_lon2,ref_lon3))
    minlon = min((ref_lon1,ref_lon2,ref_lon3))
    
    maxlat = max((ref_lat1,ref_lat2,ref_lat3))
    minlat = min((ref_lat1,ref_lat2,ref_lat3))
    
    return minlon,maxlon,minlat,maxlat      # wesn
    
def get_hgt_index(hgtlvs,mindem,maxdem):
    
    idxmin = 0
    nn = len(hgtlvs)
    for i in range(nn-1):
        if (hgtlvs[i] < mindem) & (mindem < hgtlvs[i+1]):
            minkk = i
        if (hgtlvs[i] < maxdem) & (maxdem < hgtlvs[i+1]):
            maxkk = i+1
    return minkk, maxkk
            
def kriging_levels(lonlos,latlos,hgtlvs,dwetlos,attr, maxdem, mindem, Rescale):
    
    # Get interp grids    
    lonStep = lonlos[0,1,0] - lonlos[0,0,0]
    latStep = latlos[1,0,0] - latlos[0,0,0]
    
    minlon,maxlon,minlat,maxlat = get_sar_area(attr)
    
    minlon = minlon - 0.5 #extend 0.5 degree
    maxlon = maxlon + 0.5
    minlat = minlat - 0.5
    maxlat = maxlat + 0.5
    
    lonStep = lonStep/Rescale
    latStep = latStep/Rescale
    
    lonv = np.arange(minlon,maxlon,lonStep)
    latv = np.arange(maxlat,minlat,latStep)
    
    lonvv,latvv = np.meshgrid(lonv,latv)
    
    lonvv_all = lonvv.flatten()
    latvv_all = latvv.flatten()
    
    # Get index of the useful hgtlvs
    mindex,maxdex = get_hgt_index(hgtlvs,mindem,maxdem)
    kl = np.arange(mindex,(maxdex+1))
    hgtuseful = hgtlvs[kl]
    row,col = lonvv.shape
    nh = len(kl)
    
    dwet_intp = np.zeros((row,col,nh))

    for i in range(len(kl)):
        ut.print_progress(i+1, nh, prefix='Kriging wet-delay levels:', suffix='complete', decimals=1, barLength=50, elapsed_time=None)
        lat = latlos[:,:,kl[i]]
        lon = lonlos[:,:,kl[i]]
        lat = lat.flatten()
        lon = lon.flatten()
        
        dwet0 = dwetlos[:,:,kl[i]]
        dwet0 = dwet0.flatten()
        
        dwet0_cor, para, corr= remove_ramp(lat,lon,dwet0)
        trend = func_trend_model(latvv_all,lonvv_all,para)
        
        uk = OrdinaryKriging(lon, lat, dwet0_cor, coordinates_type = 'geographic', nlags=50)
        Semivariance_trend = 2*(uk.semivariance)    
        x0 = (uk.lags)/180*np.pi*R
        y0 = Semivariance_trend
        max_length = 2/3*max(x0)
        range0 = max_length/2
        LL0 = x0[x0< max_length]
        SS0 = y0[x0< max_length]
        sill0 = max(SS0)
        sill0 = sill0.tolist()

        p0 = [sill0, range0, 0.0001]    
        vari_func = variogram_dict['spherical']        
        tt, _ = leastsq(resi_func,p0,args = (LL0,SS0))   
        corr, _ = pearsonr(SS0, vari_func(tt,LL0))
        if tt[2] < 0:
            tt[2] =0
        para = tt
        para[1] = para[1]/R/np.pi*180

        uk.variogram_model_parameters = para
        z0,s0 = uk.execute('grid', lonv, latv, n_closest_points = 15, backend='loop')
        z0 = z0.flatten() + trend
        dwet_intp[:,:,i] = z0.reshape((row,col))

    return dwet_intp, lonvv, latvv, hgtuseful


velocity_file = '/Users/caoy0a/Desktop/pyrite_test/velocity_ERA5.h5'
attr = read_attr(velocity_file)

cdic = ut.initconst()
fname0 = '/Users/caoy0a/Desktop/ERA-5_N60_N70_W25_W10_20160604_19.grb'

lvls,latlist,lonlist,gph,tmp,vpr = ut.get_ecmwf('ERA5',fname0,cdic, humidity='Q')
lonlist = lonlist - 360


hgt = np.linspace(cdic['minAltP'], gph.max().round(), cdic['nhgt'])

hgt1 = np.arange(-200, 5000, 200)
hgt2 = np.arange(5000, 10000, 500)
hgt3 = np.arange(10000, 20000, 1000)
hgt4 = np.arange(20000, gph.max().round(), 5000)


hgt11 = np.zeros(((len(hgt1)+len(hgt2)+len(hgt3)+len(hgt4)),))
hgt11[0:len(hgt1)] = hgt1[:]
hgt11[len(hgt1):(len(hgt1)+len(hgt2))] = hgt2[:]
hgt11[(len(hgt1)+len(hgt2)):(len(hgt1)+len(hgt2)+len(hgt3))] = hgt3[:]
hgt11[(len(hgt1)+len(hgt2)+len(hgt3)):len(hgt11)] = hgt4[:]

print(hgt11)


Presi,Tempi,Vpri = ut.intP2H(lvls,hgt11,gph,tmp,vpr,cdic,verbose=False)

print(Vpri[10,10,:])

ref_lat1 = 63.08203328
ref_lat2 = 63.52044068
ref_lat3 = 66.45668784
ref_lat4 = 66.92672592

ref_lon1 = -21.26566452
ref_lon2 = -16.30730990
ref_lon3 = -23.24563153
ref_lon4 = -17.62221711

width = 1685
length = 3483

lat0 = 63.2
lon0 = -19.6

lat1 = 66.6
lon1 = -21.3

lat2 = 63.4
lon2 = -21.4

lat3 = 66.2
lon3 = -23.01

lat4 = 64.06
lon4 = -16.5


lat5 = 65.9
lon5 = -17.2

orb_state_file = '/Users/caoy0a/Desktop/pyrite_test/test'

#az1,rg1 = latlon2rgaz_coarse(lat0,lon0,width,length,ref_lat1,ref_lon1,ref_lat2,ref_lon2,ref_lat3,ref_lon3,ref_lat4,ref_lon4)
rg0,az0 = latlon2rgaz_coarse(lat0,lon0,attr)
rg1,az1 = latlon2rgaz_coarse(lat1,lon1,attr)

lat6 = 64
lon6 = -20.6

uLOS, inc1 = orb_state_lalo_uLOS(lat4,lon4,orb_state_file,attr)

uLOS4, inc4 = orb_state_lalo_uLOS(lat4,lon4,orb_state_file,attr)


uLOS1, inc2  = orb_state_lalo_uLOS(lat5,lon5,orb_state_file,attr)

uLOS6, inc6 = orb_state_lalo_uLOS(lat6,lon6,orb_state_file,attr)


hN = get_geoid_point(lat6+1,lon6+1)  
print(hN)
hN = get_geoid_point(lat6,lon6)  
print(hN)

hgt = hgt + hN
hgt11 = hgt11 + hN
losX,losY,losZ,latH, lonH, losH = get_uLOS_llh(lat6,lon6,hgt11,uLOS6)
print(losH)


k0 =10

print(hgt11)

lon, lat, alt = ecef_to_lla(losX[k0],losY[k0],losZ[k0])
print(lon)
X0,Y0,Z0 = lla_to_ecef(lat, lon, alt)


print(lonlist)
#lat_intp, lon_intp, los_intp = get_LOS3D_coords(latlist,lonlist,hgt11, orb_state_file,attr)
print(lon_intp[:,:,1])
######### kriging interp ######################################
R = 6371 
#k1 = 11
lat = latlist.flatten()
lon = lonlist.flatten()

#tzd = Presi[:,:,k1]
#tzd = Vpri[:,:,k1]
#tzd = tzd.flatten()

#latvv = lat_intp[:,:,k1]
#lonvv = lon_intp[:,:,k1]

#latvv = latvv.flatten()
#lonvv = lonvv.flatten()


tzd0, para, corr= remove_ramp(lat,lon,tzd)
#trend = func_trend_model(latvv,lonvv,para)
#trend = trend.reshape(row,col)
print(lon)
uk = OrdinaryKriging(lon, lat, tzd0, coordinates_type = 'geographic', nlags=50)
Semivariance_trend = 2*(uk.semivariance)    
x0 = (uk.lags)/180*np.pi*R
y0 = Semivariance_trend

max_length = 300
range0 = max_length/2
LL0 = x0[x0< max_length]
SS0 = y0[x0< max_length]
sill0 = max(SS0)
sill0 = sill0.tolist()
    
plt.scatter(LL0, SS0)
plt.show()

def resi_func(m,d,y):
    variogram_function =variogram_dict['spherical'] 
    return  y - variogram_function(m,d)

p0 = [sill0, range0, 0.0001]    
vari_func = variogram_dict['spherical']        
tt, _ = leastsq(resi_func,p0,args = (LL0,SS0))   
corr, _ = pearsonr(SS0, vari_func(tt,LL0))
if tt[2] < 0:
    tt[2] =0
para = tt
print(para)            
            #LLm = matlab.double(LL0.tolist())
            #SSm = matlab.double(SS0.tolist()) 
            #tt = eng.variogramfit(LLm,SSm,range0,sill0,[],'nugget',0.00001,'model',inps.variogram_model)
            #variogram_para0 = tt[0]
            #para = variogram_para0[0:3]
para[1] = para[1]/R/np.pi*180
#print(para)
uk.variogram_model_parameters = para

#lonvv = lonvv+360

#z0,s0 = uk.execute('points', lonvv, latvv, n_closest_points = 15, backend='loop')
#z00,s0 = uk.execute('points', lonvv, latvv, n_closest_points = 20, backend='loop')


#Delf_dense[i,:,:] = z0 + trend

tt = tzd0.reshape((latlist.shape))

latv = latlist[:,0]
lonv = lonlist[0,:]
f = RGI((latv[::-1], lonv), tt[::-1,:],method='linear',bounds_error=False,fill_value=None)

#points = np.zeros((len(latvv),2))
#points[:,0] = latvv
#points[:,1] = lonvv 

#tzd_int = f(points)

#print(los_intp.shape)
#print(LosP.shape)
#LosP,LosT,LosV = get_LOS_parameters(latlist,lonlist,Presi,Tempi,Vpri,lat_intp, lon_intp)
cdict = ut.initconst()

#print(LosP[10,10,:])
DDry2,DWet2 = losPTV2del(LosP,LosT,LosV,los_intp ,cdict,verbose=False)

print(DDry2.shape)
print(DWet2.shape)


hgt3D = np.ones((lon_intp.shape))
for i in range(len(hgt11)):
    hgt3D[:,:,i] = hgt3D[:,:,i]*hgt11[i]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

DWet2[DWet2>1] =0
img = ax.scatter(lon_intp.flatten(), lat_intp.flatten(),hgt3D.flatten(), c=DDry2.flatten()/np.cos(30/180*np.pi), cmap=plt.hot())
#ax.set_zlim3d(0, 20000) 
cbar = fig.colorbar(img) 

plt.show()

mindex,maxdex = get_hgt_index(hgt11,150,3000)


lon3D = np.zeros((dwet_kriging.shape))
lat3D = np.zeros((dwet_kriging.shape))
hgt3D = np.ones((dwet_kriging.shape))

for i in range(dwet_kriging.shape[2]):
    lon3D[:,:,i] = lonvv
    lat3D[:,:,i] = latvv
    hgt3D[:,:,i] = hgt3D[:,:,i]*hgtuse[i]


    
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

dwet_kriging[dwet_kriging>1] =0
img = ax.scatter(lon3D.flatten(), lat3D.flatten(),hgt3D.flatten(), c=dwet_kriging.flatten(), cmap=plt.hot())
#ax.set_zlim3d(0, 20000) 
cbar = fig.colorbar(img)     
#print(dwet_kriging.shape)
#dwet_kriging = list(dwet_kriging[0])
#A = dwet_kriging[5]

#print(A.shape)
#fig, ax = plt.subplots()
#ax.imshow(A)
#plt.show()